In [43]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_aws import ChatBedrock
import re
import json

In [2]:
print("langchain advanced")

langchain advanced


In [3]:
#bedrock client 설정
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'  # replace with your region
)

In [4]:
#llm
llm = ChatBedrock(
    client=bedrock_client,
    model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
    model_kwargs={
        "max_tokens": 2000,
        "temperature": 0.15,
        "top_p": 0.9,
    }
    )

In [5]:
async def extract_stream(chain, input_json):
    collected_text = []
    async for event in chain.astream_events(input_json):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if chunk.content:
                for content_item in chunk.content:
                    if text := content_item.get('text'):
                        collected_text.append(text)
                        print(text, end="", flush=True)
    return "".join(collected_text)


In [6]:
#input { json 포맷 }  -> prompt(text로 변환) -> llm(bedrock) -> output(string타입, json타입)
#여기서 input { json 포맷 }을 활용하는 방법에 대해 이해해보자 ( 여행에 관한 정보를 얻는 llm를 만든다고 가정한다 )

In [7]:
prompt_capital = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 수도에 대해 알려줘")
])

capital_chain = prompt_capital | llm |  StrOutputParser()


# capital_chain.invoke({"country" : "태국"})

In [8]:
async for chunk in capital_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 수도는 **방콕(Bangkok)**입니다.

## 방콕에 대한 주요 정보:

### 기본 정보
- **정식 명칭**: 크룽텝마하나콘 (กรุงเทพมหานคร)
- **인구**: 약 1,000만 명 (수도권 포함)
- **면적**: 약 1,569㎢

### 주요 특징
- **정치·경제 중심지**: 태국의 정치, 경제, 문화의 중심
- **관광 명소**: 왕궁, 왓 포 사원, 왓 아룬 사원, 카오산 로드 등
- **교통**: 스카이트레인(BTS), 지하철(MRT), 보트 등 다양한 교통수단
- **음식**: 태국 전통 음식과 길거리 음식의 천국

### 문화적 특징
- 전통과 현대가 공존하는 도시
- 불교 문화가 깊이 뿌리내린 곳
- 동남아시아의 주요 허브 도시

방콕은 동남아시아에서 가장 인기 있는 관광지 중 하나이며, 풍부한 문화유산과 현대적인 도시 생활을 동시에 경험할 수 있는 매력적인 도시입니다.

In [9]:
prompt_culture = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 문화에 대해 알려줘")
])

culture_chain = prompt_culture | llm |  StrOutputParser()


# culture_chain.invoke({"country" : "태국"})


In [10]:
async for chunk in culture_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 풍부한 문화에 대해 소개해드리겠습니다.

## 🏛️ **종교와 철학**
- **불교**: 인구의 95%가 소승불교도
- 일상생활에 불교 철학이 깊이 스며들어 있음
- 전국에 4만여 개의 사원(왓)이 있음

## 🙏 **예의와 매너**
- **와이(Wai)**: 합장하며 인사하는 전통 예법
- 연장자와 지위가 높은 사람에 대한 깊은 존경
- 온화하고 미소 짓는 문화 (미소의 나라)

## 🍜 **음식 문화**
- **쌀**: 주식이며 다양한 쌀 요리
- **톰얌꿍, 팟타이, 그린커리** 등 유명 요리
- 매운맛, 신맛, 단맛, 짠맛의 조화
- 길거리 음식 문화가 발달

## 🎭 **예술과 전통**
- **전통 무용**: 콘(Khon), 라콘(Lakhon) 등
- **무에타이**: 전통 격투기
- **전통 음악**: 피파트 앙상블
- 정교한 사원 건축과 불상 조각

## 🎉 **축제와 행사**
- **송크란**: 태국 신정 (물축제)
- **로이 크라통**: 등불 축제
- **왕실 관련 행사**: 국왕에 대한 깊은 존경

## 👨‍👩‍👧‍👦 **사회 문화**
- 가족 중심의 사회
- 계층 의식이 존재하지만 온화한 성격
- "마이 펜 라이"(괜찮다) 정신 - 관용적이고 낙관적

태국 문화는 불교 정신을 바탕으로 한 평화롭고 조화로운 특징이 있습니다.

In [11]:
prompt_climate = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 날씨에 대해 알려줘")
])

climate_chain = prompt_climate | llm |  StrOutputParser()


# climate_chain.invoke({"country" : "태국"})

In [12]:
async for chunk in climate_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 날씨에 대해 설명드리겠습니다.

## 기본 특징
- **열대성 기후**: 연중 고온다습한 날씨
- **평균 기온**: 25-35°C
- **습도**: 연중 높은 습도 (70-80%)

## 계절별 특징

### 🌧️ 우기 (5월~10월)
- 몬순의 영향으로 많은 비
- 오후에 스콜성 집중호우 빈번
- 습도가 매우 높음
- 기온: 25-32°C

### ☀️ 건기 (11월~2월)
- 가장 여행하기 좋은 시기
- 비가 적고 습도 상대적으로 낮음
- 기온: 20-30°C
- 특히 12월-1월이 가장 선선

### 🔥 더위철 (3월~4월)
- 연중 가장 더운 시기
- 기온: 30-40°C
- 건조하고 매우 더움

## 지역별 차이
- **북부** (치앙마이): 상대적으로 선선, 건기/우기 구분 뚜렷
- **중부** (방콕): 전형적인 열대기후
- **남부** (푸켓, 코사무이): 연중 습하고 따뜻, 지역별로 우기 시기 다름

## 여행 팁
- **최적 시기**: 11월-2월 (건기)
- **준비물**: 우산, 선크림, 가벼운 옷, 에어컨 대비 얇은 겉옷

In [13]:
prompt_location = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 유명한 관광 명소에 대해 알려줘")
])

location_chain = prompt_location | llm |  StrOutputParser()


# climate_location.invoke({"country" : "태국"})

In [14]:
async for chunk in location_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 주요 관광 명소들을 지역별로 소개해드리겠습니다.

## 🏛️ 방콕 (Bangkok)
- **왓 포 (Wat Pho)** - 거대한 와불상으로 유명한 사원
- **왕궁 (Grand Palace)** - 태국 왕실의 화려한 궁전
- **왓 아룬 (Wat Arun)** - 새벽 사원으로 불리는 아름다운 사원
- **차오프라야 강** - 롱테일 보트 투어
- **카오산 로드** - 배낭여행자들의 성지

## 🏖️ 푸켓 (Phuket)
- **파통 비치** - 나이트라이프와 해변 액티비티
- **피피 섬** - 영화 '더 비치' 촬영지
- **빅 부다** - 45m 높이의 거대한 불상
- **올드 타운** - 중국-포르투갈 스타일 건축물

## 🌴 끄라비 (Krabi)
- **라일레이 비치** - 암벽등반과 에메랄드빛 바다
- **홍 섬** - 석회암 절벽으로 둘러싸인 라군
- **타이거 케이브 템플** - 1,237개 계단을 올라가는 사원

## 🏝️ 코사무이 (Koh Samui)
- **빅 부다 템플** - 12m 높이의 황금 불상
- **나무앙 폭포** - 열대우림 속 아름다운 폭포
- **차웽 비치** - 가장 인기 있는 해변

## 🏛️ 치앙마이 (Chiang Mai)
- **도이 수텝** - 산 위의 황금 사원
- **올드 시티** - 고대 성벽으로 둘러싸인 구시가지
- **나이트 바자** - 전통 수공예품 쇼핑
- **코끼리 보호구역** - 윤리적 코끼리 체험

## 🏺 아유타야 (Ayutthaya)
- **아유타야 역사공원** - UNESCO 세계문화유산
- **왓 마하탓** - 나무뿌리에 감싸인 부처 머리상
- **방파인 궁전** - 여름 별궁

각 지역마다 독특한 매력이 있으니, 관심사와 여행 스타일에 맞춰 선택하시면 됩니다!

In [15]:
#여행에 관한 정보를 얻는 llm를 만든다고 가정한다 위의 네가지 정보를 모두 포함하여 전체적인 형태의 여행 정보를 얻고 싶다고 가정 할때 어떻게 langchain에 활용할수 있을까?
#결국 llm에서 결과가 나오면 string임으로 이 string을 json안에 assgin가능하다

In [16]:
# input에 {"country": "태국"}이라고 두고 itemgetter("country")를 통해 변수를 가져온뒤 각각 llm의 결과를 얻고 sting으로 json에 포함시킨다면? 
# llm을 통과한 결과값들이 새로운 input(json타입)을 형성할수 있지 않을까?
new_input = {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | location_chain
            }

In [17]:
# new_input의 capital, culture, climate, location가  template_total의 capital, culture, climate, location 변수와 연결된다
template_total = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}

이 정보를 바탕으로 여행 가기 가장 좋은 계절을 알려주세요
"""
prompt_total  = PromptTemplate(input_variables=["capital", "culture", "climate", "location"],
                                 template=template_total)

In [18]:
travel_chain = new_input | prompt_total | llm | StrOutputParser()

In [19]:
async for chunk in travel_chain.astream({"country": "태국"}):
    print(chunk, end="")

제가 제공한 태국 기후 정보를 바탕으로 말씀드리면, **11월부터 2월까지가 태국 여행의 최적 시기**입니다! 🌟

## 🎯 가장 좋은 시기: 11월-2월 (건기)

### 왜 이 시기가 최고인가요?
- **쾌적한 기온**: 20-30°C로 연중 가장 선선함
- **낮은 습도**: 다른 시기 대비 상대적으로 건조
- **적은 강수량**: 비 걱정 없이 야외 활동 가능
- **맑은 하늘**: 사진 찍기에도 완벽!

### 특히 12월-1월이 베스트! ⭐
- 가장 선선하고 쾌적한 날씨
- 크리스마스, 신정 휴가철과 겹쳐 여행하기 좋음
- 모든 관광지에서 최상의 컨디션

## 지역별 추천 시기

### 🏛️ 방콕 & 중부 지역
- **11월-2월**: 왕궁, 사원 관광에 최적
- 야외 시장 구경도 덜 덥고 쾌적

### 🏖️ 남부 해변 지역 (푸켓, 끄라비)
- **11월-3월**: 해변 액티비티와 섬 투어에 완벽
- 바다가 잔잔하고 맑음

### 🏛️ 북부 (치앙마이)
- **11월-2월**: 도이 수텝 등반, 트레킹에 이상적
- 밤에는 약간 쌀쌀할 수 있어 얇은 겉옷 필요

## 💡 여행 팁
이 시기는 성수기라 **항공료와 숙박비가 비싸고 예약이 빨리 찬다**는 점 참고하세요! 미리미리 계획하시는 것을 추천드립니다.

특정 지역이나 액티비티 계획이 있으시다면 더 구체적인 조언을 드릴 수 있어요! 🇹🇭

In [20]:
#비슷한 방법으로 RunnableParallel도 존재한다 
parallel_chain = RunnableParallel(
                                  capital=capital_chain, 
                                  culture=culture_chain,
                                  climate=climate_chain, 
                                  location =location_chain
                                 )

In [21]:
travel_chain_parallel =  parallel_chain | prompt_total | llm | StrOutputParser()

In [22]:
async for chunk in travel_chain_parallel.astream({"country": "태국"}):
    print(chunk, end="")


제가 제공한 태국 날씨 정보를 바탕으로 말씀드리면, **11월부터 2월까지의 건기**가 태국 여행하기에 가장 좋은 계절입니다! 🌞

## 건기 (11월-2월)를 추천하는 이유:

### 🌡️ 쾌적한 기온
- 평균 기온이 25-30°C로 태국 기준에서는 가장 시원함
- 40°C까지 올라가는 더위철(3-5월)에 비해 훨씬 견딜만함

### 💧 낮은 습도
- 상대적으로 습도가 낮아 덜 끈적거림
- 우기(6-10월)의 70-80% 습도보다 훨씬 쾌적

### ☀️ 맑은 날씨
- 거의 매일 맑은 하늘을 볼 수 있음
- 우기처럼 매일 스콜성 비가 내리지 않아 야외활동 최적

### 🏖️ 관광지별 장점
- **방콕**: 왕궁, 사원 관광하기 좋은 날씨
- **푸켓/끄라비**: 해변 액티비티와 섬 투어 최적
- **치앙마이**: 도이 수텝 등 산악 관광 쾌적
- **코사무이**: 맑은 바다에서 수상스포츠 즐기기 좋음

다만 이 시기는 성수기라 항공료와 숙박비가 비싸고 관광지가 붐빌 수 있다는 점은 참고하세요! 그래도 날씨만큼은 정말 완벽한 시기입니다. ✈️

In [23]:
# llm의 outpu의 경우 StrOutputParser()를 활용해서 string으로 나오지만 llm.with_structured_output(obj)을 통해 json포맷으로 받을수 있다 
# llm의 결과가 json포맷으로 오면 이를 활용하여 다른 langchain에 활용할수 있다

In [24]:
template_plan = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}

이 정보를 바탕으로 여행 계획을 세워주세요
"""
prompt_plan  = PromptTemplate(input_variables=["capital", "culture", "climate", "location"],
                                 template=template_plan)

In [25]:
#llm의 결과값 포맷의 schema
class Travel(BaseModel):
    capital: str = Field(description="국가의 수도에 대한 설명")
    culture: str = Field(description="국가의 문화에 대한 설명")
    climate : str = Field(description="국가 기후에 대한 설명")
    location : str = Field(description="국가 관광명소에 대한 설명")
    plan: str = Field(description="여행계획")


In [26]:
one_chain = (
    {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | location_chain
    }
    | prompt_plan
    | llm.with_structured_output(Travel)
)
 

In [27]:
async for chunk in one_chain.astream({"country": "태국"}):
    print(chunk, end="")


capital='태국의 수도는 방콕(Bangkok)입니다. 정식 명칭은 크룽텝마하나콘이며, 인구 약 1,000만 명의 대도시입니다. 태국의 정치, 경제, 문화의 중심지로 왕궁, 왓 포 사원, 왓 아룬 사원, 카오산 로드 등 주요 관광명소가 있습니다. 스카이트레인(BTS), 지하철(MRT), 보트 등 다양한 교통수단이 발달되어 있으며, 전통 태국 문화와 현대 문명이 조화롭게 공존하는 동남아시아의 주요 허브 도시입니다.' culture="태국은 인구의 95%가 소승불교도인 불교 국가로, 일상생활에 불교 철학이 깊이 스며들어 있습니다. 와이(Wai) 합장 인사법으로 존경을 표현하며, 연장자와 지위가 높은 사람을 깊이 존경하는 문화입니다. '미소의 나라'라 불릴 만큼 온화하고 미소 짓는 문화를 가지고 있습니다. 쌀이 주식이며 톰얌꿍, 팟타이, 그린커리 등 세계적으로 유명한 요리와 발달된 길거리 음식 문화를 자랑합니다. 무에타이 전통 무술, 우아한 전통 무용, 송크란 물축제, 로이 끄라통 등불 축제 등 풍부한 전통 예술과 축제 문화를 가지고 있으며, 왕실에 대한 깊은 존경과 사랑을 보입니다." climate='태국은 열대성 기후로 연중 고온다습하며 평균 기온은 25-35°C, 습도는 70-80%입니다. 건기(11월~2월)는 가장 여행하기 좋은 시기로 기온 25-30°C, 상대적으로 낮은 습도와 맑은 날씨를 보입니다. 더위철(3월~5월)은 가장 더운 시기로 기온이 30-40°C까지 올라가며 매우 덥고 건조합니다. 우기(6월~10월)는 몬순 시즌으로 기온 25-32°C, 거의 매일 스콜성 비가 내리고 습도가 매우 높습니다. 지역별로는 북부가 상대적으로 서늘하고, 중부는 전형적인 열대기후, 남부는 연중 습하고 따뜻한 특징을 보입니다.' location='태국의 주요 관광명소는 다음과 같습니다. 방콕에는 거대한 와불상의 왓 포, 새벽 사원 왓 아룬, 화려한 그랜드 팰리스, 차오프라야 강, 배낭여행자 성지 카오산 로드가 있습니다. 푸켓에는 파통 비치, 영화 촬영지 피피 섬, 4

In [28]:
#data는 llm의 결과 값을 의미합니다  RunnableLambda함수를 활용하면 data와 llm결과를 연결 시켜줄수 있습니다
def transform_keys(data):
    """Transform keys from history to chat_history"""
    return {
        "capital":  data.capital,
        "culture":  data.culture,  
        "climate":  data.climate, 
        "location": data.location, 
        "plan":     data.plan
    }

In [29]:
template_detail = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}
plan :      {plan}

plan에 담겨있는 관광지 혹은 축제에 대한 정보를 좀더 상세하게 알려두세요 
"""
prompt_detail  = PromptTemplate(input_variables=["capital", "culture", "climate", "location", "plan"],
                                 template=template_detail)

In [30]:
# RunnableLambda(transform_keys)는 llm과 prompt_detail의 데이터를 매핑 시켜주는 역할
chain_and_chain = (
    {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | location_chain
    }
    | prompt_plan
    | llm.with_structured_output(Travel)
    | RunnableLambda(transform_keys)
    | prompt_detail
    | llm
    | StrOutputParser()
)


In [31]:
async for chunk in chain_and_chain.astream({"country": "일본"}):
    print(chunk, end="")


안녕하세요! 일본 여행 계획에 포함된 관광지와 축제에 대해 더 자세히 알려드리겠습니다.

**도쿄의 현대적 도시 문화 관광지:**

도쿄 스카이트리는 높이 634m로 세계에서 두 번째로 높은 전파탑입니다. 350m와 450m 높이에 전망대가 있어 도쿄 전경을 한눈에 볼 수 있어요. 시부야 교차로는 하루 약 300만 명이 지나가는 세계에서 가장 붐비는 교차로로, 특히 야간에 네온사인과 함께 보는 광경이 장관입니다.

**교토의 전통 문화 관광지:**

기요미즈데라는 778년에 건립된 유네스코 세계문화유산으로, 나무못 하나 사용하지 않고 지어진 본당의 무대가 유명합니다. 후시미 이나리 신사는 1만 개가 넘는 붉은 도리이가 산 정상까지 이어져 있어 신비로운 터널을 만들어냅니다. 아라시야마 대나무 숲은 수천 그루의 대나무가 만드는 초록빛 터널로, 바람이 불 때 나는 소리가 '일본의 음풍경 100선'에 선정되기도 했습니다.

**계절별 주요 축제:**

봄에는 벚꽃축제(사쿠라 마츠리)가 3월 말부터 5월 초까지 전국에서 열립니다. 여름에는 7월 교토의 기온 마츠리, 8월 도쿄의 스미다가와 불꽃축제가 대표적이에요. 가을에는 단풍축제와 함께 각 지역 신사에서 수확감사 축제들이 열리고, 겨울에는 삿포로 눈축제(2월)와 각종 일루미네이션 축제들이 도시를 화려하게 장식합니다.

이런 정보들이 여행 계획 세우시는 데 도움이 되시나요?

In [32]:
template_plan_final = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}

이 정보를 바탕으로 여행 계획을 세워주세요
"""
prompt_plan_final  = PromptTemplate(input_variables=["capital", "culture", "climate"],
                                 template=template_plan_final)

In [33]:
class TravelLast(BaseModel):
    capital: str = Field(description="국가의 수도에 대한 설명")
    culture: str = Field(description="국가의 문화에 대한 설명")
    climate : str = Field(description="국가 기후에 대한 설명")
    plan: str = Field(description="여행계획")

In [34]:
def transform_key_last(data):
    """Transform keys from history to chat_history"""
    return {
        "capital":  data.capital,
        "culture":  data.culture,  
        "climate":  data.climate, 
        "plan":     data.plan
    }

In [35]:
template_detail_final = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
plan :      {plan}

plan에 담겨있는 관광지 혹은 축제에 대한 정보를 좀더 상세하게 알려두세요 
"""
prompt_detail_final = PromptTemplate(input_variables=["capital", "culture", "climate", "plan"],
                                 template=template_detail_final)

In [36]:
two_json_chain=({
    "capital": itemgetter("country") | capital_chain,
    "culture": itemgetter("country") | culture_chain,
    "climate": itemgetter("country") | climate_chain
    }
    | prompt_plan_final
    | llm.with_structured_output(TravelLast)
    | RunnableLambda(transform_key_last)
    | {
        "capital": itemgetter("capital"),   # capital("country") -> itemgetter("capital")
        "culture": itemgetter("culture"),   # 쉼표 추가
        "climate": itemgetter("climate"),   # 쉼표 추가
        "plan": itemgetter("plan")
    }
    | prompt_detail_final
    | llm
    | StrOutputParser()
    )


In [37]:
async for chunk in two_json_chain.astream({"country": "중국"}):
    print(chunk, end="")


중국 여행 계획에 포함된 주요 관광지들에 대해 상세히 설명해드리겠습니다!

## **베이징 주요 관광지**

**자금성(紫禁城)**
- 명·청 황제들의 궁궐로 1420년 완공, 약 600년 역사
- 9,999개 방을 가진 세계 최대 규모의 목조 건축군
- 태화전, 중화전, 보화전 등 주요 전각에서 황실 문화 체험
- 입장료: 60위안(성수기 4-10월), 40위안(비수기)

**만리장성 팔달령 구간**
- 베이징에서 70km, 가장 잘 보존된 구간
- 명나라 때 건설된 성벽으로 험준한 산세를 따라 구불구불
- 케이블카 이용 가능, 등반 시간 2-3시간
- 입장료: 45위안, 케이블카 왕복 140위안

**천단공원(天壇)**
- 명·청 황제들이 하늘에 제사 지내던 제단
- 기년전(祈年殿)의 독특한 원형 건축이 유명
- 새벽 6-8시에 현지인들의 태극권, 검무 연습 관람 가능

## **시안 주요 관광지**

**병마용 박물관**
- 진시황릉의 부장품으로 1974년 발견
- 8,000여 개의 병사, 말, 전차 등 실물 크기 토용
- 1,2,3호 갱과 청동마차 전시관으로 구성
- 입장료: 150위안, 관람 시간 3-4시간 권장

**시안 성벽**
- 명나라 때 건설된 중국 최대 규모 성벽 (둘레 13.7km)
- 자전거 대여로 성벽 일주 가능 (2-3시간 소요)
- 야간 조명으로 아름다운 야경 연출

**회족거리(回民街)**
- 이슬람 문화가 살아있는 전통 거리
- 양러우파오모(양고기 빵죽), 량피(냉면) 등 특색 음식
- 200여 개 상점과 식당이 밀집

## **상하이 주요 관광지**

**와이탄(外灘)**
- 황푸강변 1.5km 구간의 서구식 건축군
- 대한민국 임시정부 청사, 평화호텔 등 역사적 건물
- 밤 8시 루자쭈이 마천루 야경이 장관

**예원(豫園)**
- 명나라 관리가 부모를 위해 조성한 강남식 정원
- 400년 역사의 전통 건축과 조경 예술의 걸작
- 인근 예원상성에서 전통 공예품과 간식 구매 가능

## **항저우/소주 주요 관광지**

**서

In [45]:
template_plan_streaming = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}

이 정보를 바탕으로 여행 계획을 세워주세요

응답은 반드시 다음 JSON 형식으로 제공해주세요:
{{
    "capital": "수도 정보",
    "culture": "문화 정보", 
    "climate": "기후 정보",
    "plan": "여행 계획"
}}
"""
prompt_plan_streaming  = PromptTemplate(input_variables=["capital", "culture", "climate"],
                                 template=template_plan_streaming)

In [46]:
info_chain = RunnableParallel({
    "capital": itemgetter("country") | capital_chain,
    "culture": itemgetter("country") | culture_chain,
    "climate": itemgetter("country") | climate_chain
})

# 2단계: 계획 생성
plan_chain = prompt_plan_streaming | llm | StrOutputParser()

# 3단계: 최종 상세 계획
detail_chain = prompt_detail_final | llm | StrOutputParser()


In [ ]:
async def execute_with_progress(country):
    # 1단계 실행 + 출력
    info = await info_chain.ainvoke({"country": country})
    print("✅ 기본 정보 수집 완료:", info)
    
    # 2단계 스트리밍 실행
    print("✅ 여행 계획 생성 중...")
    plan_text = ""
    async for chunk in plan_chain.astream(info):
        print(chunk, end="")
        plan_text += chunk
    
    # JSON 파싱
    json_match = re.search(r'\{.*\}', plan_text, re.DOTALL)
    if json_match:
        plan_data = json.loads(json_match.group())
        plan = type('Plan', (), plan_data)()
    else:
        # 파싱 실패시 기본 구조
        plan = type('Plan', (), {'capital': '', 'culture': '', 'climate': '', 'plan': plan_text.strip()})()
    
    print("\n✅ 여행 계획 생성 완료")
    
    # 3단계 실행
    transformed = transform_key_last(plan)
    async for chunk in detail_chain.astream(transformed):
        print(chunk, end="")


In [ ]:
await execute_with_progress("중국")

✅ 기본 정보 수집 완료: {'capital': "한국의 수도는 **서울특별시**입니다.\n\n## 서울에 대한 주요 정보:\n\n### 기본 정보\n- **정식 명칭**: 서울특별시 (Seoul Special City)\n- **면적**: 약 605㎢\n- **인구**: 약 950만 명 (2023년 기준)\n- **행정구역**: 25개 자치구\n\n### 역사\n- 1394년 조선 태조가 한양으로 천도한 이후 600년 이상 수도 역할\n- 일제강점기에는 '경성'으로 불림\n- 1945년 해방 후 '서울'로 개칭\n- 1988년 서울올림픽, 2002년 FIFA 월드컵 공동개최\n\n### 주요 특징\n- **정치**: 대한민국 정부기관 소재지\n- **경제**: 국내 GDP의 약 20% 차지하는 경제 중심지\n- **교육**: 서울대, 연세대, 고려대 등 주요 대학 위치\n- **문화**: K-pop, 드라마 등 한류 문화의 중심지\n- **교통**: 지하철 9개 노선, 인천국제공항과 연결\n\n### 주요 명소\n- 경복궁, 창덕궁 등 조선왕조 궁궐\n- 남산타워, 한강공원\n- 명동, 강남, 홍대 등 상업지구\n\n서울은 전통과 현대가 조화를 이루는 국제적인 대도시입니다.", 'culture': '한국 문화는 오랜 역사와 전통을 바탕으로 현대적 요소가 조화롭게 어우러진 독특한 특징을 가지고 있습니다.\n\n## 🏛️ 전통 문화\n- **유교 문화**: 효도, 예의, 존경 등을 중시하는 가치관\n- **한복**: 아름다운 곡선미의 전통 의상\n- **한옥**: 자연과 조화를 이루는 전통 건축\n- **전통 예술**: 판소리, 탈춤, 사물놀이 등\n\n## 🍚 음식 문화\n- **김치**: 발효 음식의 대표작\n- **한정식**: 다양한 반찬이 함께 나오는 식사 문화\n- **공유 문화**: 음식을 나누어 먹는 문화\n- **계절 음식**: 절기에 따른 특별한 음식들\n\n## 🎵 현대 문화 (한류)\n- **K-POP**: BTS, 블랙핑크 등 전 세계적 인